<a href="https://colab.research.google.com/github/EdervanPsilva/ML_AnaliseSatisfacao/blob/main/ML_AnaliseSatisfacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Código em Python utilizando Machine Learning para aprendizado e análises de satisfações em comentários.

#Importando Bibliotecas

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from textblob import TextBlob
import random

#Dataframe para aprendizado

In [ ]:
data = {
    'text': [
        "Gostei",
        "Muito ruim!",
        "Gostei demais",
        "Mais ou menos, da pra melhorar!",
        "O melhor!",
        "Ruim, não gostei",
        "Muito bom!",
        "Um pouco lento, mas eu gostei.",
        "Horrível",
        "Muito bom, gostei demais",
        "Bom",
        "Ruim",
        "Gostei",
        "Muito bom!",
        "Gostei.",
        "Horrível demais",
        "Muito bom",
        "Não recomendo!"


],
    'label': [
        1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0
    ]
}

df = pd.DataFrame(data)

# Adicionando novas frases do dataframe

In [ ]:
import random

# Função para gerar frases aleatórias
def gerar_frase(sentimento):
  palavras_positivas = ["bom", "ótimo", "excelente", "incrível", "fantástico", "adorei", "amei", "perfeito", "recomendo", "eficiente","Gostei"]
  palavras_negativas = ["ruim", "péssimo", "horrível", "terrível", "decepcionante", "frustrante", "odeio", "não gostei", "lento", "complicado"]

  palavras = palavras_positivas if sentimento == 1 else palavras_negativas
  num_palavras = random.randint(3, 8)
  frase = " ".join(random.choice(palavras) for _ in range(num_palavras))
  return frase.capitalize() + "!"

# Gerar novas frases e labels
novas_frases = []
for _ in range(200):
  novas_frases.append((gerar_frase(1), 1))  # Boas avaliações
  novas_frases.append((gerar_frase(0), 0))  # Ruins avaliações

# Adicionar as novas frases ao dataframe
novas_linhas = pd.DataFrame(novas_frases, columns=['text', 'label'])
data = pd.concat([pd.DataFrame(data), novas_linhas], ignore_index=True)

#Treinamento do modelo ML

In [ ]:
# Pré-processamento usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text'])

# Remover valores None do rótulo para treinamento
data_train = data[data['label'].notna()]
X_train = vectorizer.transform(data_train['text'])
y_train = data_train['label'].astype(int)

# Treinamento do modelo Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Predição para todos os comentários, incluindo o novo
predicted_labels = model.predict(X)

# Adicionar a coluna 'predicted_label' ao DataFrame
data['predicted_label'] = predicted_labels
data['predicted_label'] = data['predicted_label'].apply(lambda x: 'Positivo' if x == 1 else 'Negativo')

# Atualizar a coluna 'label' do novo comentário com a previsão
data.loc[data['label'].isna(), 'label'] = data.loc[data['label'].isna(), 'predicted_label'].apply(lambda x: 1 if x == 'Positivo' else 0)


#Novo dataframe para ser analisado

In [ ]:
# Novo DataFrame para ser analisado
new_data = {
    'text': [
        "Bom demais",
        "Não gostei, achei muito lento",
        "Muito bom, recomendo",
        "Muito legal",
        "Não Gostei"



    ]
}

new_df = pd.DataFrame(new_data)



In [ ]:
# Pré-processamento do novo DataFrame
X_new = vectorizer.transform(new_df['text'])

# Predição para o novo DataFrame
new_predictions = model.predict(X_new)

# Adicionar a coluna 'Resultado' ao novo DataFrame
new_df['Resultado'] = new_predictions
new_df['Resultado'] = new_df['Resultado'].apply(lambda x: 'Positivo ✔' if x == 1 else 'Negativo ❌')

print(new_df)

                            text   Resultado
0                     Bom demais  Positivo ✔
1  Não gostei, achei muito lento  Negativo ❌
2           Muito bom, recomendo  Positivo ✔
3                    Muito legal  Positivo ✔
4                     Não Gostei  Negativo ❌
